In [1]:
from torchview import draw_graph
import torch
from transformers import MistralForSequenceClassification, LlamaTokenizerFast

import graphviz
graphviz.set_jupyter_format('png')

'svg'

In [2]:
hf_repo = 'h2oai/h2o-danube2-1.8b-base'
device = 'cpu'

In [3]:
model = MistralForSequenceClassification.from_pretrained(hf_repo, num_labels=3, device_map=device,
                                                         torch_dtype=torch.bfloat16)

tokenizer = LlamaTokenizerFast.from_pretrained(hf_repo)

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at h2oai/h2o-danube2-1.8b-base and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:

if tokenizer.pad_token is None:
    num_added_toks = tokenizer.add_special_tokens({'pad_token': '<|pad_token|>'})
    assert num_added_toks == 1
    model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id


In [5]:

premises = [
    "and",
]

hypotheses = [
    'my'
]

inputs = tokenizer(premises, hypotheses, truncation='only_first', padding="max_length",
                    max_length=25, add_special_tokens=True, return_tensors='pt')

inputs

{'input_ids': tensor([[32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000,   304,   586]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         1]])}

In [6]:


model_graph = draw_graph(model, input_data=inputs, save_graph=True, filename=f"graphed_hf_repo.svg", directory='/mnt/c/Users/lachl/PycharmProjects/bart-large-mnli-recreation')


In [7]:
type(model_graph.visual_graph)

graphviz.graphs.Digraph

In [8]:

# fie_ext = 'png'
# temp_img = 'temp_file'
# my_graph = graphviz.Source(model_graph)
# my_graph.render(temp_img,format=fie_ext, view=False)

In [10]:
model_graph.visual_graph

AttributeError: 'Digraph' object has no attribute 'show'